In [1]:
import pandas as pd
import numpy as np
import json
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import html

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36 Edg/89.0.774.45'}
source = requests.get("https://kabuoji3.com/stock/1305/", headers=headers).text
soup = BeautifulSoup(source, "html.parser")
table = soup.find_all("tbody")


In [24]:
print(table)

[<tbody>
<tr>
<td>2021-03-09</td>
<td>2004</td>
<td>2019</td>
<td>1993</td>
<td>2018</td>
<td>342760</td>
<td>2018</td>
</tr>
</tbody>, <tbody>
<tr>
<td>2020-06-30</td>
<td>1674</td>
<td>1675</td>
<td>1653</td>
<td>1653</td>
<td>13350</td>
<td>1653</td>
</tr>
</tbody>]


In [3]:
driver = webdriver.Chrome(
    executable_path=r'C:\Users\KOJ\PycharmProjects\untitled\GitMaster\auto_Investment\chromedriver')
driver.get("https://kabuoji3.com/stock/1305/")
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')
table = soup.find_all("table", class_='stock_table stock_data_table')
df = pd.read_html(str(table), header=0)

In [30]:
print(df[0])

             日付    始値    高値    安値    終値     出来高  終値調整
0    2021-03-09  2004  2019  1993  2018  342760  2018
1    2021-03-03  1999  2005  1990  2004  179760  2004
2    2021-02-26  1997  1997  1953  1953  523910  1953
3    2021-02-24  2031  2034  2001  2001   45260  2001
4    2021-02-12  2039  2040  2028  2031   59500  2031
..          ...   ...   ...   ...   ...     ...   ...
146  2020-07-07  1674  1679  1666  1672   97870  1672
147  2020-07-06  1654  1678  1654  1677   63840  1677
148  2020-07-03  1649  1652  1639  1649   32180  1649
149  2020-07-02  1644  1650  1635  1639   83860  1639
150  2020-07-01  1659  1660  1632  1635  553730  1635

[151 rows x 7 columns]


In [4]:
len(df)

2

In [32]:
result = pd.concat([df[0], df[1]])

In [33]:
print(result)

             日付    始値    高値    安値    終値     出来高  終値調整
0    2021-03-09  2004  2019  1993  2018  342760  2018
1    2021-03-03  1999  2005  1990  2004  179760  2004
2    2021-02-26  1997  1997  1953  1953  523910  1953
3    2021-02-24  2031  2034  2001  2001   45260  2001
4    2021-02-12  2039  2040  2028  2031   59500  2031
..          ...   ...   ...   ...   ...     ...   ...
144  2019-11-25  1786  1789  1783  1786   83690  1786
145  2019-11-22  1772  1781  1771  1772   49880  1772
146  2019-11-21  1767  1773  1747  1770  244080  1770
147  2019-11-20  1772  1780  1766  1772   67770  1772
148  2019-11-19  1777  1783  1774  1778   29100  1778

[300 rows x 7 columns]


In [34]:
result = result.drop("終値調整", axis=1)
result = result.rename(columns={'日付':'Date', '始値':'Open', '高値':'High', '安値':'Low', '終値':'Close', '出来高':'Volume'})

In [35]:
print(result)

           Date  Open  High   Low  Close  Volume
0    2021-03-09  2004  2019  1993   2018  342760
1    2021-03-03  1999  2005  1990   2004  179760
2    2021-02-26  1997  1997  1953   1953  523910
3    2021-02-24  2031  2034  2001   2001   45260
4    2021-02-12  2039  2040  2028   2031   59500
..          ...   ...   ...   ...    ...     ...
144  2019-11-25  1786  1789  1783   1786   83690
145  2019-11-22  1772  1781  1771   1772   49880
146  2019-11-21  1767  1773  1747   1770  244080
147  2019-11-20  1772  1780  1766   1772   67770
148  2019-11-19  1777  1783  1774   1778   29100

[300 rows x 6 columns]


In [88]:
s = requests.Session()
for cookie in driver.get_cookies():
    s.cookies.set(cookie['name'], cookie['value'])

In [89]:
ETF_Link = pd.read_csv('ETF_link.csv', encoding='utf-8')

In [90]:
ETF_Link = ETF_Link.values

In [91]:
etfNum = ETF_Link[0][0].split('.')[-2].split('=')[-1]
timeSeriesLink = "https://s20.si0.kabu.co.jp/ap/PC/InvInfo/Market/StockDetail/Default?Symbol=" + etfNum + "&Exchange=TSE&Category=PRICE_HISTORY"
result = s.get(timeSeriesLink).text
soup = BeautifulSoup(result, "html.parser")
table = soup.find_all("table", class_="tb3")
df = pd.read_html(str(table), header=0)[0]


In [92]:
df = df.drop("前日比", axis=1)
df = df.rename(columns={'基準日':'Date', '始値':'Open', '高値':'High', '安値':'Low', '終値':'Close', '出来高':'Volume'})

In [93]:
def fix_date(row):
    return row.split('(')[0]
df['Date'] = df['Date'].apply(fix_date)

In [94]:
df = df.replace('--', np.nan)
df = df.dropna(axis=0)

In [95]:
print(df)

          Date   Open   High    Low  Close Volume
0   2021/03/05  13130  13260  13130  13260     69
1   2021/03/04  13190  13190  13130  13190     21
2   2021/03/03  13120  13190  13120  13190     11
3   2021/03/02  13170  13170  13170  13170     10
4   2021/03/01  13060  13120  13060  13120      6
5   2021/02/26  13210  13210  13060  13060     21
6   2021/02/25  13100  13200  13100  13200     88
7   2021/02/24  13120  13130  13060  13100    200
8   2021/02/22  13090  13160  13090  13140    112
9   2021/02/19  13200  13230  13130  13130     20
10  2021/02/18  13290  13290  13200  13200     33
11  2021/02/17  13230  13290  13210  13290     79
12  2021/02/16  13190  13230  13190  13230    327
13  2021/02/15  13120  13190  13120  13190     62
14  2021/02/12  13110  13150  13110  13110     29
15  2021/02/10  13170  13170  13110  13110     62
16  2021/02/09  13130  13190  13110  13170     18
17  2021/02/08  13040  13140  13040  13140     14
18  2021/02/05  13180  13180  13060  13160     40


/ap/PC/InvInfo/Market/StockDetail/ListCsv?SymbolCode=1349&SymbolName=%EF%BC%A1%EF%BC%A2%EF%BC%A6%E6%B1%8E%E3%82%A2%E3%82%B8%E3%82%A2%E5%82%B5%E5%88%B8%E3%82%A4%E3%83%B3%E3%83%87%E3%83%83%E3%82%AF%E3%82%B9%E3%83%BB%E3%83%95%E3%82%A1%E3%83%B3%E3%83%89&exchange=TSE&exchangeName=%E6%9D%B1%E4%BA%AC%20ETF%2FETN
